In [65]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time

In [115]:
holdings=['RHHBY','BIG','DOCU','CLX']

In [116]:
#ticker, #1wkret #1wkmean, 4wk, 4wkmean, kchange, kchangemean,k_change percentile, std, stdmean
res=[[],[],[],[],[],[],[],[],[],[]]

In [119]:
for ticker in holdings:
    print(ticker)
    d={'function': 'TIME_SERIES_WEEKLY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )
    spy=[[],[],[],[],[],[],[]]
    for key, val in response.json()['Weekly Adjusted Time Series'].items():
        spy[0].append(float(val['1. open']))
        spy[1].append(float(val['2. high']))
        spy[2].append(float(val['3. low']))
        spy[3].append(float(val['4. close']))
        spy[4].append(float(val['5. adjusted close']))
        #spy[5].append(float(val['6. volume']))
        #spy[6].append(float(val['7. dividend amount']))
    t=list(response.json()['Weekly Adjusted Time Series'].keys())
    time.sleep(1)
    try:
        if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
            print('Notlive')
            d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'False','chartLast':'1'}
            response = requests.get(
            'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format(ticker)#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
            ,params=d,
            )
            
            spy[0].insert(0,float(response.json()[0]['open']))
            spy[1].insert(0,float(response.json()[0]['high']))
            spy[2].insert(0,float(response.json()[0]['low']))
            spy[3].insert(0,float(response.json()[0]['close']))
            spy[4].insert(0,float(response.json()[0]['close']))
            t.insert(0,str(response.json()[0]['date']))

            #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
    except:
        print('hi')

        
        
    spy=pd.DataFrame({'Date':t,
                 'open':spy[0],
                  'high':spy[1],
                  'low':spy[2],
                  'close':spy[3],
                  'adj_close':spy[4]#,
                #  'volume':spy[5],
                #  'dividend':spy[6],
                 })
    spy=spy.astype({'Date': 'datetime64'})
    spy=spy.sort_values(by='Date')   
    spy['ret4wk']=spy.adj_close/spy.adj_close.shift(3)-1
    spy['highest_high']=spy.high.rolling(12).max()
    spy['lowest_low']=spy.low.rolling(12).min()
    spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
    spy['k']=spy.k.rolling(3).mean()
    spy['k_change'] = spy.k/spy.k.shift(3)-1
    spy['rstd']=spy.adj_close.rolling(12).std()
    spy['ret1wk']=spy.adj_close/spy.adj_close.shift(1)-1
    d={'ticker':ticker,'k_change':spy.k_change.values[-1]}
    res[0].append(ticker)
    res[1].append(spy.ret1wk.values[-1])
    res[2].append(spy.ret1wk.rolling(3).mean().values[-1])
    res[3].append(spy.ret4wk.values[-1])
    res[4].append(spy.ret4wk.rolling(3).mean().values[-1])
    res[5].append(spy.k_change.values[-1])
    res[6].append(spy.k_change.rolling(3).mean().values[-1])
    res[7].append(np.nanpercentile(spy.k_change,q=10))
    res[8].append(spy.rstd.values[-1])
    res[9].append(spy.rstd.rolling(3).mean().values[-1])
    
    
    
    

RHHBY
Notlive
hi
BIG
Notlive
DOCU
Notlive
CLX
Notlive


In [120]:
res=pd.DataFrame({'ticker':res[0], 
            'ret1wk':res[1],
             'ret1wk_mean':res[2],
             'ret4wk':res[3],
             'ret4wk_mean':res[4],
             'k_change':res[5],
             'k_change_mean':res[6],
             'k_change_percentile':res[7],
             'rstd':res[8],
             'rstd_mean':res[9]
             }
            )

In [121]:
res.sort_values(by='ret1wk_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
1,BIG,0.000308,0.076395,0.241485,0.327787,0.261087,0.575977,-0.599538,7.328717,6.376447
2,DOCU,-0.000451,0.045356,0.140528,0.207788,0.056751,0.053010,-0.373238,21.012217,18.700637
0,RHHBY,-0.019952,0.014570,0.043417,0.029566,0.077959,0.173695,-0.521244,2.876446,2.748540
3,CLX,-0.000453,-0.008565,-0.027199,0.034639,0.155184,0.277629,-0.487275,13.439147,14.180736


In [94]:
res.sort_values(by='ret1wk_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k_change,k_change_mean,rstd,rstd_mean
2,DOCU,0.028190,0.025119,0.077123,0.047594,-0.041221,-0.022630,5.213754,5.555341
1,BIG,0.027595,0.023021,0.070565,0.121235,-0.024565,0.000481,2.898700,2.649084
0,GDX,-0.033229,-0.004576,-0.014932,0.014822,-0.174732,0.021306,1.194316,1.178216
3,CLX,-0.022718,-0.007053,-0.021329,-0.023248,-0.597027,-0.407580,2.994666,2.897018


In [89]:
res.sort_values(by='ret1wk_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k_change,k_change_mean,rstd,rstd_mean
2,MBUU,-0.021702,0.035501,0.105023,0.212375,0.367513,0.418247,4.642872,4.469681
1,AZN,0.029935,0.012033,0.036294,0.014843,-0.062579,-0.245810,0.781145,0.714343
0,SPY,-0.005153,0.000521,0.001356,0.021040,0.268086,0.749622,5.008670,4.658034
3,MSFT,-0.008349,-0.000480,-0.001592,0.010671,0.110782,0.205791,2.064047,2.186029
4,NFLX,-0.032412,-0.014429,-0.042910,-0.012232,-0.365573,-0.104620,7.978609,8.842592


In [9]:
msft=spy.copy(deep=True)

In [14]:
msft['ret4wk']=msft.adj_close/msft.adj_close.shift(3)-1
spy['ret4wk']=spy.adj_close/spy.adj_close.shift(3)-1

In [16]:
spy['highest_high']=spy.high.rolling(12).max()
spy['lowest_low']=spy.low.rolling(12).min()
spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
spy['k']=spy.k.rolling(3).mean()
spy['k_change'] = spy.k/spy.k.shift(3)-1

msft['highest_high']=msft.high.rolling(12).max()
msft['lowest_low']=msft.low.rolling(12).min()
msft['k']=(msft.close-msft.lowest_low)/(msft.highest_high-msft.lowest_low)
msft['k']=msft.k.rolling(3).mean()
msft['k_change'] = msft.k/msft.k.shift(3)-1




In [22]:
spy['rstd']=spy.adj_close.rolling(12).std()
msft['rstd']=msft.adj_close.rolling(12).std()

In [25]:
msft['ret1wk']=msft.adj_close/msft.adj_close.shift(1)-1
spy['ret1wk']=spy.adj_close/spy.adj_close.shift(1)-1

In [26]:
msft.tail(10)

,Date,open,high,low,close,adj_close,volume,dividend,ret4wk,highest_high,lowest_low,k,k_change,rstd,ret1wk
9,2020-03-20,140.00,150.15,135.0000,137.35,137.35,421347734.0,0.0,-0.152213,190.70,135.00,0.226001,-0.641828,12.881111,-0.135239
8,2020-03-27,137.01,156.66,132.5200,149.70,149.70,358866845.0,0.0,-0.073467,190.70,132.52,0.242002,-0.424578,13.538858,0.089916
7,2020-04-03,152.44,164.78,150.0100,153.83,153.83,290191457.0,0.0,-0.031480,190.70,132.52,0.234586,-0.239293,13.876028,0.027589
6,2020-04-09,160.32,170.00,157.5800,165.14,165.14,229630744.0,0.0,0.202330,190.70,132.52,0.407414,0.802706,13.859286,0.073523
5,2020-04-17,164.35,180.00,162.3000,178.60,178.60,238965670.0,0.0,0.193053,190.70,132.52,0.572992,1.367711,14.477445,0.081507
4,2020-04-24,176.63,178.75,166.1102,174.55,174.55,194621072.0,0.0,0.134694,190.70,132.52,0.691704,1.948616,14.676007,-0.022676
3,2020-05-01,176.59,180.40,169.3900,174.57,174.57,212119968.0,0.0,0.057103,190.70,132.52,0.745732,0.830404,13.910374,0.000115
2,2020-05-08,174.49,185.00,173.8000,184.68,184.68,158579959.0,0.0,0.034043,188.18,132.52,0.794096,0.385877,13.888795,0.057914
1,2020-05-15,183.15,187.51,175.6800,183.16,183.16,196126640.0,0.0,0.049327,187.51,132.52,0.860257,0.243678,14.351698,-0.008230
0,2020-05-19,185.75,186.60,183.4900,183.63,183.63,61664938.0,0.0,0.051899,187.51,132.52,0.929152,0.245960,15.227043,0.002566


In [38]:
spy.ret4wk.rolling(3).mean().values[-1]
spy.k_change.rolling(3).mean().values[-1]
spy.rstd.rolling(3).mean().values[-1]

22.84021899844733

In [47]:
spy.ret1wk.values[-1]

0.019875646220483656